In [ ]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
)
import time

from azure.ai.ml.dsl import pipeline
from azure.ai.ml import Input

import ast

In [ ]:
def fetch_data(model_name):
       import requests
       import pandas as pd
       from io import StringIO
       from sklearn.model_selection import train_test_split
       from sklearn.preprocessing import LabelEncoder

       seed = 2020

       url = f""
       response = requests.get(url)

       response.raise_for_status()

       csv_data = StringIO(response.text)

       df = pd.read_csv(csv_data)

       test_size, val_size = .2, .2

       train_df, temp_df = train_test_split(df, test_size=test_size + val_size, random_state=seed, stratify=df['label_string'])

       val_size_adj = val_size / (val_size + test_size)
       val_df, test_df = train_test_split(temp_df, test_size=1 - val_size_adj, random_state=seed, stratify=temp_df['label_string'])

       train_df.to_json("train.jsonl", orient='records', lines=True)
       val_df.to_json("val.jsonl", orient='records', lines=True)
       test_df.to_json("test.jsonl", orient='records', lines=True)
